In [1]:
import torch as th
import syft as sy

In [2]:
sy.sandbox(globals(), False, False)

Setting up Sandbox...
Done!


In [43]:
# positive, non-negative, single entitiy, overlapping, symmetric mult
x = (
    th.tensor([0])
    .int()
    .track_sensitivity(
        min_ent_conts=th.tensor([[0, 0, 0, 0]]), max_ent_conts=th.tensor([[3, 0, 0, 0]])
    )
)


y = x.clamp_min(1)

assert y.sensitivity == th.tensor([2])
assert y.max_vals == th.tensor([3])
assert y.min_vals == th.tensor([1])
assert (y.entities == th.tensor([[1, 0, 0, 0]])).all()

# negative, non-positive, single entitiy, overlapping, symmetric mult
x = (
    th.tensor([-2])
    .int()
    .track_sensitivity(
        min_ent_conts=th.tensor([[-3, 0, 0, 0]]), max_ent_conts=th.tensor([[0, 0, 0, 0]])
    )
)

y = x.clamp_min(-2)

assert y.sensitivity == th.tensor([2])
assert y.max_vals == th.tensor([0])
assert y.min_vals == th.tensor([-2])
assert (y.entities == th.tensor([[1, 0, 0, 0]])).all()

# negative, positive, single entitiy, overlapping, symmetric mult
x = (
    th.tensor([-1])
    .int()
    .track_sensitivity(
        min_ent_conts=th.tensor([[-3, 0, 0, 0]]), max_ent_conts=th.tensor([[3, 0, 0, 0]])
    )
)

y = x.clamp_min(-2)

assert y.sensitivity == th.tensor([5])
assert y.max_vals == th.tensor([3])
assert y.min_vals == th.tensor([-2])
assert (y.entities == th.tensor([[1, 0, 0, 0]])).all()

# negative, positive, dual entitiy, overlapping, symmetric mult
x = (
    th.tensor([3])
    .int()
    .track_sensitivity(
        min_ent_conts=th.tensor([[-3, 0, 0, 0]]), max_ent_conts=th.tensor([[2, 1, 0, 0]])
    )
)


In [44]:
y = x.clamp_min(1)

In [48]:
x.child.min_ent_conts

tensor([[-3,  0,  0,  0]])

In [33]:
min_ = x.child.min_ent_conts.clamp_min(1) * x.child.entities
min_

tensor([[1, 1, 0, 0]])

In [34]:
x.child.expanded_max_vals - x.child.max_ent_conts

tensor([[1, 2, 3, 3]])

In [36]:
max_ = x.child.max_ent_conts.clamp_min(1) * x.child.entities
max_

tensor([[2, 1, 0, 0]])

In [37]:
# negative, positive, dual entitiy, overlapping, symmetric mult
x = (
    th.tensor([2])
    .int()
    .track_sensitivity(
        min_ent_conts=min_, max_ent_conts=max_
    )
)

In [10]:
x.max_vals

tensor([2])

In [11]:
x.child.max_ent_conts.sum(1)

tensor([3])

In [12]:
x.min_vals

tensor([1])

In [13]:
x.child.entities

tensor([[1, 0, 0, 0]])

In [20]:
y = x.clamp_min(1)

ValuesOutOfSpecifiedMinMaxRangeException: You tried to initialize a SensitivityTensor whose values were outsideof the specified min and max range. Most likely this means your min andmax range are incorrect.

In [5]:


assert y.sensitivity == th.tensor([2])
assert y.max_vals == th.tensor([2]) # TODO: this should return 1 but we don't have smart enough logic for that yet
assert y.min_vals == th.tensor([0])
assert (y.entities == th.tensor([[1, 1, 0, 0]]).byte()).all()



ValuesOutOfSpecifiedMinMaxRangeException: You tried to initialize a SensitivityTensor whose values were outsideof the specified min and max range. Most likely this means your min andmax range are incorrect.

In [6]:













# negative, positive, dual entitiy, non-overlapping, non-symmetric add
x = (
    th.tensor([1])
    .int()
    .track_sensitivity(
        min_ent_conts=th.tensor([[-1, 0, 0, 0]]), max_ent_conts=th.tensor([[1, 1, 0, 0]])
    )
)

y = (
    th.tensor([5])
    .int()
    .track_sensitivity(
        min_ent_conts=th.tensor([[4, 0, 0, 0]]), max_ent_conts=th.tensor([[5, 5, 0, 0]])
    )
)

z = x == y

assert z.sensitivity == th.tensor([1]).byte()
assert z.max_vals == th.tensor([2]) # TODO: this should return 1 but we don't have smart enough logic for that yet
assert z.min_vals == th.tensor([0])
assert (z.entities == th.tensor([[1, 1, 0, 0]]).byte()).all()

ValuesOutOfSpecifiedMinMaxRangeException: You tried to initialize a SensitivityTensor whose values were outsideof the specified min and max range. Most likely this means your min andmax range are incorrect.

In [36]:
z.max_vals

tensor([2])